### train_kd()


*   Knowledge Distillation(KD) 학습을 위한 한 epoch의 학습 루프를 정의
*   student model(`model`)을 학습시키되, teacher model (`teacher_model`)의 출력을 참고하여 KD loss (`loss_fn_kd`)를 사용해 파라미터를 업데이트하고 학습 중 손실(loss)과 성능 지표(metrics)를 추적·기록하는 함수



1.   Student 모델은 train mode, Teacher 모델은 eval mode로 설정
2.   각 mini-batch에 대해:
        *   student의 예측값 계산
        *   teacher의 예측값 계산 (gradient 없음)
        *   label + teacher output을 함께 사용하는 KD loss 계산
        *   backpropagation 및 optimizer step
3.    일정 step마다:
        *   metric 계산 (accuracy 등)
        *   loss 평균 추적
4.    epoch 전체에 대한 평균 metric을 logging





In [ ]:
# Defining train_kd & train_and_evaluate_kd functions
def train_kd(model, teacher_model, optimizer, loss_fn_kd, dataloader, metrics, params):
    """
    한 epoch 동안 Knowledge Distillation 방식으로 student model을 학습하는 함수

    Args:
        model: (torch.nn.Module) student 모델
        teacher_model: (torch.nn.Module) teacher 모델
        optimizer: (torch.optim) student 모델의 optimizer
        loss_fn_kd: KD loss 함수 (student output, label, teacher output 사용)
        dataloader: 학습용 데이터 로더
        metrics: (dict) 평가 지표 함수들 (accuracy 등)
        params: (Params) 하이퍼파라미터 모음
    """

    # set model to training mode
    # student 모델을 학습 모드로 설정 (dropout, batchnorm 활성화)
    model.train()
    # teacher 모델을 평가 모드로 설정 (파라미터 고정, dropout 비활성화)
    teacher_model.eval()

    # summary for current training loop and a running average object for loss
    # summary 저장용 리스트 (metric 기록)
    summ = []
    # loss의 이동 평균을 계산하기 위한 객체
    loss_avg = utils.RunningAverage()

    # Use tqdm for progress bar
    # tqdm을 이용해 진행 상황(progress bar) 표시
    with tqdm(total=len(dataloader)) as t:
        # dataloader에서 batch 단위로 데이터 로드
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            # move to GPU if available
            # CUDA 사용 시 GPU로 데이터 이동
            if params.cuda:
                train_batch, labels_batch = train_batch.cuda(async=True), \
                                            labels_batch.cuda(async=True)
            # convert to torch Variables
            # Tensor를 Variable로 감싸서 autograd 사용
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)

            # compute model output, fetch teacher output, and compute KD loss
            # student 모델의 출력 계산
            output_batch = model(train_batch)

            # get one batch output from teacher_outputs list
            # teacher 모델의 출력 계산 (gradient 계산 안 함)
            with torch.no_grad():
                output_teacher_batch = teacher_model(train_batch)
            # CUDA 사용 시 teacher output도 GPU로 이동
            if params.cuda:
                output_teacher_batch = output_teacher_batch.cuda(async=True)

            # KD loss 계산
            # (student output, true label, teacher output, temperature/alpha 등 params 사용)
            loss = loss_fn_kd(output_batch, labels_batch, output_teacher_batch, params)

            # clear previous gradients, compute gradients of all variables wrt loss
            # 이전 step의 gradient 초기화
            optimizer.zero_grad()
            # loss에 대한 gradient 계산 (backpropagation)
            loss.backward()

            # performs updates using calculated gradients
            # optimizer를 통해 student 모델 파라미터 업데이트
            optimizer.step()

            # Evaluate summaries only once in a while
            # 일정 step마다 metric 계산 및 기록
            if i % params.save_summary_steps == 0:
                # extract data from torch Variable, move to cpu, convert to numpy arrays
                # Tensor → CPU → NumPy 변환
                output_batch = output_batch.data.cpu().numpy()
                labels_batch = labels_batch.data.cpu().numpy()

                # compute all metrics on this batch
                # 각 metric 계산
                summary_batch = {metric:metrics[metric](output_batch, labels_batch)
                                 for metric in metrics}
                # 현재 batch의 loss 값 저장
                summary_batch['loss'] = loss.data[0]
                # summary 리스트에 추가
                summ.append(summary_batch)

            # update the average loss
            # loss의 이동 평균 업데이트
            loss_avg.update(loss.data[0])

            # tqdm에 현재 평균 loss 출력
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            # progress bar 한 step 진행
            t.update()

    # compute mean of all metrics in summary
    # epoch 전체에 대해 metric 평균 계산
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]}
    # logging을 위한 문자열 생성
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    # 학습 metric 로그 출력
    logging.info("- Train metrics: " + metrics_string)

### loss_fn_kd


*   `loss_fn_kd` 함수는 Knowledge Distillation(KD) 학습에서 사용하는 손실 함수를 정의
*   이 함수는 두 가지 손실을 결합한 총 loss를 계산


1.   Soft target loss (KD loss)


        *   student 모델의 출력과 teacher 모델의 출력 분포를
        *   temperature T를 적용한 KL Divergence로 비교


2.   Hard target loss (일반 supervised loss)

        *   student 모델의 출력과 정답 label 사이의
        *   Cross Entropy loss

*   이 두 손실을 α(alpha) 로 가중합하여 최종 loss를 만듭니다.






In [ ]:
def loss_fn_kd(outputs, labels, teacher_outputs, params):
    """
    Knowledge Distillation(KD) loss를 계산하는 함수

    Args:
        outputs: student 모델의 출력 logits
        labels: 정답 label (hard target)
        teacher_outputs: teacher 모델의 출력 logits
        params: 하이퍼파라미터 객체 (alpha, temperature 포함)

    NOTE:
    PyTorch의 KLDivLoss는 입력으로 'log-probability'를 기대하므로
    student 출력에는 log_softmax를 사용해야 함
    """

    # KD loss에서 soft target과 hard target의 비중을 조절하는 계수
    alpha = params.alpha

    # temperature: softmax 분포를 얼마나 부드럽게 할지 결정
    T = params.temperature

    # Knowledge Distillation loss 계산
    KD_loss = (
        # (1) Soft target loss: teacher와 student 분포 간 KL Divergence
        nn.KLDivLoss()(
            # student 출력: temperature로 나눈 뒤 log_softmax 적용
            F.log_softmax(outputs / T, dim=1),

            # teacher 출력: temperature로 나눈 뒤 softmax 적용
            F.softmax(teacher_outputs / T, dim=1)
        ) * (alpha * T * T)   # 논문에서 제안된 scaling (gradient 보정 목적)

        # (2) Hard target loss: 일반적인 supervised cross entropy
        + F.cross_entropy(outputs, labels) * (1. - alpha)
    )

    # 최종 KD loss 반환
    return KD_loss